Cargar archivo movies.CSV a dataframe, Ver las columnas del DataFrame, Verificar y eliminar valores nulos

In [ ]:
import pandas as pd
import ast

dfMovies = pd.read_csv(r'C:\Users\pc\Downloads\movies_dataset.csv')

print(dfMovies.info())
print(dfMovies.columns)            
print(dfMovies.isnull().sum())
dfMovies.dropna(inplace=True)

Preprocesamiento de datos: tratamiento de datos antes de realizar la desanidacion. Se realiza una funcion que crea una copia del Df original para no afectarlo, posteriormente itera sobre cada columna del dataFrame verificando si la columna es de tipo objeto (cadenas) para en ese caso convertirlas a listas o diccionarios. 

In [4]:
#conversion a listas o diccionarios: 
def convertir_list_dic(df):
    df_converted = df.copy()
    
    for column in df_converted.columns:
        if df_converted[column].dtype == 'object':
            try:
                df_converted[column] = df_converted[column].apply(ast.literal_eval)
            except (ValueError, SyntaxError):
                print(f"Error al convertir la columna '{column}'. Las cadenas no tienen el formato correcto.")
    
    return df_converted

df = pd.DataFrame(dfMovies)
df_converted = convertir_list_dic(dfMovies)

print(df_converted)

Error al convertir la columna 'homepage'. Las cadenas no tienen el formato correcto.
Error al convertir la columna 'imdb_id'. Las cadenas no tienen el formato correcto.
Error al convertir la columna 'original_language'. Las cadenas no tienen el formato correcto.
Error al convertir la columna 'original_title'. Las cadenas no tienen el formato correcto.
Error al convertir la columna 'overview'. Las cadenas no tienen el formato correcto.
Error al convertir la columna 'popularity'. Las cadenas no tienen el formato correcto.
Error al convertir la columna 'poster_path'. Las cadenas no tienen el formato correcto.
Error al convertir la columna 'release_date'. Las cadenas no tienen el formato correcto.
Error al convertir la columna 'status'. Las cadenas no tienen el formato correcto.
Error al convertir la columna 'tagline'. Las cadenas no tienen el formato correcto.
Error al convertir la columna 'title'. Las cadenas no tienen el formato correcto.
Error al convertir la columna 'video'. Las caden

In [5]:
# Se crea un ciclo for para comprobar que tipo de elementos tienen las columnas. 
for column in df_converted.columns:
    if df_converted[column].dtype == 'object':
        are_lists = df_converted[column].apply(lambda x: isinstance(x, list)).all()
        are_dicts = df_converted[column].apply(lambda x: isinstance(x, dict)).all()
        
        if are_lists:
            print(f"\nTodos los elementos en la columna '{column}' son listas.")
        elif are_dicts:
            print(f"\nTodos los elementos en la columna '{column}' son diccionarios.")
        else:
            print(f"\nLa columna '{column}' contiene elementos mixtos o no es una lista ni un diccionario.")



Todos los elementos en la columna 'belongs_to_collection' son diccionarios.

Todos los elementos en la columna 'genres' son listas.

La columna 'homepage' contiene elementos mixtos o no es una lista ni un diccionario.

La columna 'imdb_id' contiene elementos mixtos o no es una lista ni un diccionario.

La columna 'original_language' contiene elementos mixtos o no es una lista ni un diccionario.

La columna 'original_title' contiene elementos mixtos o no es una lista ni un diccionario.

La columna 'overview' contiene elementos mixtos o no es una lista ni un diccionario.

La columna 'popularity' contiene elementos mixtos o no es una lista ni un diccionario.

La columna 'poster_path' contiene elementos mixtos o no es una lista ni un diccionario.

Todos los elementos en la columna 'production_companies' son listas.

Todos los elementos en la columna 'production_countries' son listas.

La columna 'release_date' contiene elementos mixtos o no es una lista ni un diccionario.

Todos los eleme

Desanidacion de datos: Se crea un funcion que revisa cada columna y si esta contiene listas usa .explode() para expandirlas, si la columna contiene diccionarios usa pd.jason_normalize() para descomponer 
cada diccionario en columnas individuales y las renombra para evitar duplicados, finalmente las columnas desanidadas se agregan al DataFrame y la columna original se elimina.

In [6]:
def desanidar_columnas(df):
    
    for column in df.columns:
        if df[column].apply(lambda x: isinstance(x, list)).any():
            df = df.explode(column).reset_index(drop=True)
        
        if df[column].apply(lambda x: isinstance(x, dict)).any():
            desanidado_df = pd.json_normalize(df[column])
            desanidado_df.columns = [f"{column}_{subcol}" for subcol in desanidado_df.columns]
            df = pd.concat([df.reset_index(drop=True), desanidado_df], axis=1).drop(columns=[column])
    
    return df

df_desanidado = desanidar_columnas(df_converted)
print(df_desanidado.columns)

Index(['adult', 'budget', 'homepage', 'id', 'imdb_id', 'original_language',
       'original_title', 'overview', 'popularity', 'poster_path',
       'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title',
       'video', 'vote_average', 'vote_count', 'belongs_to_collection_id',
       'belongs_to_collection_name', 'belongs_to_collection_poster_path',
       'belongs_to_collection_backdrop_path', 'genres_id', 'genres_name',
       'production_companies_name', 'production_companies_id',
       'production_countries_iso_3166_1', 'production_countries_name',
       'spoken_languages_iso_639_1', 'spoken_languages_name'],
      dtype='object')


In [7]:
print("\nNúmero de filas antes y después de desanidar:")
print(f"Filas originales: {len(df)}")
print(f"Filas después de desanidar: {len(df_desanidado)}")



Número de filas antes y después de desanidar:
Filas originales: 693
Filas después de desanidar: 15771


In [8]:
#cambiar valores nulos a cero
print(df_desanidado['revenue'].isnull().sum())
dfMovies['revenue'] = dfMovies['revenue'].fillna(0)

0


In [9]:
#se agrega .astype() para indicar que debe ser float el tipo de dato
print(df_desanidado['budget'].isnull().sum())
dfMovies['budget'] = dfMovies['budget'].fillna(0)
dfMovies['budget'] = dfMovies['budget'].astype(float)

0


In [10]:
#valores nulos de release date deben eliminarse, y se usa reset_index para restablecer el indice del dataframe para evitar mantener los indices anteriores  
df_cleaned = df_desanidado.dropna(subset=['release_date']).reset_index(drop=True)
print("\nDataFrame después de eliminar valores nulos en la columna 'release_date':")
print(df_cleaned['release_date'])


DataFrame después de eliminar valores nulos en la columna 'release_date':
0        1995-11-16
1        1995-11-16
2        1995-11-16
3        1995-11-16
4        1995-11-16
            ...    
15766    2017-05-12
15767    2017-05-12
15768    2017-05-12
15769    2017-05-12
15770    2017-05-12
Name: release_date, Length: 15771, dtype: object


#fechas en formato AAAA-mm-dd y columna release_year 

In [11]:
#para la tranformacion primero se uso el argumento dayFirst=True para priorizar el dia sobre mes en fechas ambiguas,
# se ordenan las fechas y posteriormente se les dio el formato AAAA-mm-dd. 

df_cleaned['release_date'] = pd.to_datetime(df_cleaned['release_date'], dayfirst=True)
df_cleaned = df_cleaned.sort_values(by='release_date')
df_cleaned['release_date'] = pd.to_datetime(df_cleaned['release_date'], format='%d/%m/%Y')
print(df_cleaned['release_date'])

532     1940-11-13
533     1940-11-13
534     1940-11-13
4070    1942-05-27
4072    1942-05-27
           ...    
15700   2017-07-11
15699   2017-07-11
15698   2017-07-11
15697   2017-07-11
15696   2017-07-11
Name: release_date, Length: 15771, dtype: datetime64[ns]


C:\Users\pc\AppData\Local\Temp\ipykernel_26916\3428505200.py:4: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df_cleaned['release_date'] = pd.to_datetime(df_cleaned['release_date'], dayfirst=True)


#crear columna release_year

In [12]:
df_cleaned['release_year'] = df_cleaned['release_date'].dt.year
print(df_cleaned['release_year'])

532      1940
533      1940
534      1940
4070     1942
4072     1942
         ... 
15700    2017
15699    2017
15698    2017
15697    2017
15696    2017
Name: release_year, Length: 15771, dtype: int32


#crear columna con retorno de inversion: se usa .apply() para aplicar una función lambda a cada fila del DataFrame, la cual calcula el retorno de inversion y se indica que si el valor de budget es 0, asigna 0 al retorno de inversion.

In [13]:
df_cleaned['return'] = df_cleaned.apply(lambda row: row['revenue'] / row['budget'] if row['budget'] != 0 else 0,axis=1)
print(df_cleaned['return'])

532      36.543860
533      36.543860
534      36.543860
4070      0.000000
4072      0.000000
           ...    
15700     2.433605
15699     2.433605
15698     2.433605
15697     2.433605
15696     2.433605
Name: return, Length: 15771, dtype: float64


In [14]:
print(df_cleaned.columns)

Index(['adult', 'budget', 'homepage', 'id', 'imdb_id', 'original_language',
       'original_title', 'overview', 'popularity', 'poster_path',
       'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title',
       'video', 'vote_average', 'vote_count', 'belongs_to_collection_id',
       'belongs_to_collection_name', 'belongs_to_collection_poster_path',
       'belongs_to_collection_backdrop_path', 'genres_id', 'genres_name',
       'production_companies_name', 'production_companies_id',
       'production_countries_iso_3166_1', 'production_countries_name',
       'spoken_languages_iso_639_1', 'spoken_languages_name', 'release_year',
       'return'],
      dtype='object')


In [19]:

columnas_a_eliminar = [ 'adult','video','imdb_id','original_title','poster_path','homepage']
df_cleaned = df_cleaned.drop(columns=columnas_a_eliminar)
print(df_cleaned.columns)

Index(['budget', 'id', 'original_language', 'overview', 'popularity',
       'release_date', 'revenue', 'runtime', 'status', 'title', 'vote_average',
       'vote_count', 'belongs_to_collection_id', 'belongs_to_collection_name',
       'belongs_to_collection_poster_path',
       'belongs_to_collection_backdrop_path', 'genres_id', 'genres_name',
       'production_companies_name', 'production_companies_id',
       'production_countries_iso_3166_1', 'production_countries_name',
       'spoken_languages_iso_639_1', 'spoken_languages_name', 'release_year',
       'return'],
      dtype='object')


In [20]:
print(df_cleaned['spoken_languages_name'])

532      English
533      English
534      English
4070     English
4072     English
          ...   
15700    English
15699    English
15698    English
15697    English
15696    English
Name: spoken_languages_name, Length: 15771, dtype: object


In [21]:
print(f"Numero de filas: {len(df_cleaned)}")

Numero de filas: 15771


In [22]:
print(df_cleaned['release_date'])

532     1940-11-13
533     1940-11-13
534     1940-11-13
4070    1942-05-27
4072    1942-05-27
           ...    
15700   2017-07-11
15699   2017-07-11
15698   2017-07-11
15697   2017-07-11
15696   2017-07-11
Name: release_date, Length: 15771, dtype: datetime64[ns]


In [30]:
#Debido a que la columna "popularity" contiene valores que pandas intenta convertir a tipo numerico al querer transformar a formato parquet, 
#pero algunos valores estan en formato string se realiza la conversion previa para que no me arroje error al momento de la conversion. 
df_cleaned['popularity'] = pd.to_numeric(df_cleaned['popularity'], errors='coerce')
print(df_cleaned['popularity'])

532        8.037960
533        8.037960
534        8.037960
4070       0.476623
4072       0.476623
            ...    
15700    146.161786
15699    146.161786
15698    146.161786
15697    146.161786
15696    146.161786
Name: popularity, Length: 15771, dtype: float64


Para las funciones solicitadas se crea un DF con los campos: 'title','release_date','release_year','popularity','spoken_languages_name','vote_count','vote_average'. A los cuales se les realizan modificacones que buscan mejorar el rendimiento y eficiencia en memoria y ajustarse a las distintas especificaciones que se solicitaban. 

In [32]:
df_funciones = df_cleaned[['title','release_date','release_year','popularity','spoken_languages_name','vote_count','vote_average']].copy() #copia del df original 
df_funciones = df_funciones[
    (df_funciones['release_date'].dt.year >= 1980) & #Se filtran a peliculas de los ochentas a la actualidad para optimizar espacio en memoria y traer titulos mas actuales. 
    (df_funciones['release_year'] >= 1980) &         #Se busca lo mismo que en la linea pasada. 
    (df_funciones['spoken_languages_name'] == 'English') & #Limite el df a solo peliculas donde el idioma sea en ingles. 
    (df_funciones['vote_count'] >= 1000)                   #Tambien se eliminaron aquellas peliculas cuyos votos estaba por debajo de 2000 ya que de igual la funcion "votos_titulo" no las tomara en cuenta. 
].reset_index(drop=True)

print(df_funciones.head())
print(f"Numero de filas: {len(df_funciones)}")

                     title release_date  release_year  popularity  \
0  The Empire Strikes Back   1980-05-17          1980   19.470959   
1  The Empire Strikes Back   1980-05-17          1980   19.470959   
2  The Empire Strikes Back   1980-05-17          1980   19.470959   
3  The Empire Strikes Back   1980-05-17          1980   19.470959   
4  The Empire Strikes Back   1980-05-17          1980   19.470959   

  spoken_languages_name  vote_count  vote_average  
0               English      5998.0           8.2  
1               English      5998.0           8.2  
2               English      5998.0           8.2  
3               English      5998.0           8.2  
4               English      5998.0           8.2  
Numero de filas: 5161


In [38]:
#Se convierte a archivo.parquet para optimizacion de espacio y velocidad.  
df_funciones.to_parquet("df_funciones.parquet", engine='pyarrow', index=False)
df_verificado = pd.read_parquet("df_funciones.parquet")
print(df_verificado.head())

                     title release_date  release_year  popularity  \
0  The Empire Strikes Back   1980-05-17          1980   19.470959   
1  The Empire Strikes Back   1980-05-17          1980   19.470959   
2  The Empire Strikes Back   1980-05-17          1980   19.470959   
3  The Empire Strikes Back   1980-05-17          1980   19.470959   
4  The Empire Strikes Back   1980-05-17          1980   19.470959   

  spoken_languages_name  vote_count  vote_average  
0               English      5998.0           8.2  
1               English      5998.0           8.2  
2               English      5998.0           8.2  
3               English      5998.0           8.2  
4               English      5998.0           8.2  


Se crea una funcion donde se ingresa un mes en idioma Español y devuelve la cantidad de películas que fueron estrenadas en el mes.
consultado en el Dataset "df_funciones".

In [34]:
# primero se crea un diccionario para traducir los meses en español a su número correspondiente, luego se convierte el mes ingresado en español a minúsculas y se obtiene su número.
# Finalmente se filtran las películas que fueron estrenadas en el mes consultado.

def cantidad_filmaciones_mes(mes):
    ""
    meses = {
        'enero': 1, 'febrero': 2, 'marzo': 3, 'abril': 4, 'mayo': 5, 'junio': 6,
        'julio': 7, 'agosto': 8, 'septiembre': 9, 'octubre': 10, 'noviembre': 11, 'diciembre': 12
    }
    
    mes = mes.lower()
    mes_numero = meses.get(mes)
    
    if mes_numero is None:
        return "El mes ingresado no es válido. Por favor, ingrese un mes en español."
    cantidad = df_funciones[df_funciones['release_date'].dt.month == mes_numero].shape[0]
    
    return f"{cantidad} películas fueron estrenadas en el mes de {mes.capitalize()}."


print(cantidad_filmaciones_mes("febrero"))

507 películas fueron estrenadas en el mes de Febrero.


Se crea una funcion donde se ingresa un día en idioma Español el cual devuelve la cantidad de películas que fueron estrenadas en día.
consultado en el Dataset "df_funciones".

In [35]:
#Se crea un diccionario para traducir los días de la semana en español a su número correspondiente (lunes=0, domingo=6).
#Se busca convertir el día ingresado en español a minúsculas y obtener su número correspondiente.
#Por ultimo se filtran las películas que fueron estrenadas en el día consultado.
def cantidad_filmaciones_dia(dia):
    
    dias_semana = {
        'lunes': 0, 'martes': 1, 'miércoles': 2, 'jueves': 3, 
        'viernes': 4, 'sábado': 5, 'domingo': 6
    }
    dia = dia.lower()
    dia_numero = dias_semana.get(dia)
    
    if dia_numero is None:
        return "El día ingresado no es válido. Por favor, ingrese un día de la semana en español."
    cantidad = df_funciones[df_funciones['release_date'].dt.dayofweek == dia_numero].shape[0]
    return f"{cantidad} películas fueron estrenadas en los días {dia.capitalize()}."


print(cantidad_filmaciones_dia("sábado"))

201 películas fueron estrenadas en los días Sábado.


La funcion "score_titulo" busca que al ingresar el título de una filmación arroje como respuesta el título, el año de estreno y el score.
En esta funcion se modifico el df para la columna "titulo" donde remplace los espacios por "_", para que al momento de realizar la busqueda en el 
servidor lo pueda detectar y no de error. 

In [51]:
#Se filtra el df para encontrar la filmación con el título ingresado, posteriormente se comprueba si se encontró la película.
#Despues se obtiene la informacion necesaria de los campos 'title','release_year','popularity' para dar el resultado solicitado. 
def score_titulo(titulo_de_la_filmacion: str):
   
    titulo_formateado = titulo_de_la_filmacion.lower().replace("_", " ")
    pelicula = df_funciones[df_funciones['title'].str.lower() == titulo_formateado]
    
    if pelicula.empty:
        return "No se encontró una película con ese título."
    
    
    titulo = pelicula['title'].values[0]
    año = pelicula['release_year'].values[0]
    score = pelicula['popularity'].values[0]  

    return f"La película '{titulo}' fue estrenada en el año {año} con un score/popularidad de {score}."

print(score_titulo("The Empire Strikes Back"))

La película 'The Empire Strikes Back' fue estrenada en el año 1980 con un score/popularidad de 19.470959.


En la siguiente funcion se ingresa el título de una filmación esperando como respuesta el título, la cantidad de votos y el valor promedio de las votaciones. 
Como esta variable debe contar con al menos 2000 valoraciones para cumplir con los requerimientos, el df se ajusto a que tomara peliculas con votaciones 
por encima de 1000 valoraciones para mostrar el mensaje avisando que no cumple esta condición y que por ende, no se devuelve ningun valor.

In [57]:
#Se filtra el Df reemplazando espacios con guiones bajos en el título ingresado para poder buscarla dese el servidor
#despues se verifica si la pelicula fue encontrada. En caso de que si, se obtiene el título, año en que se estreno, 
#cantidad de votos y promedio de votos para posteriormente verificar si cumple con al menos 2000 valoraciones.
#en ambos casos arroja un mensaje si cumplio o no con la peticion. 

def votos_titulo(titulo_de_la_filmacion):
    
    titulo_formateado = titulo_de_la_filmacion.replace("_", " ").lower()
    pelicula = df_funciones[df_funciones['title'].str.lower() == titulo_formateado]
    if pelicula.empty:
        return "No se encontró una película con ese título."

    titulo = pelicula['title'].values[0]
    año = pelicula['release_year'].values[0]
    votos = pelicula['vote_count'].values[0]
    promedio_votos = pelicula['vote_average'].values[0]
    
    if votos < 2000:
        return f"La película '{titulo}' no cumple con el mínimo de 2000 valoraciones (actual: {votos})."
    
    return f"La película '{titulo}' se estrenó en el año {año}, cuenta con un total de {votos} valoraciones, con un promedio de {promedio_votos}."

print(votos_titulo("Alien:_Covenant"))


La película 'Alien: Covenant' se estrenó en el año 2017, cuenta con un total de 2677.0 valoraciones, con un promedio de 5.7.


In [ ]:
#transformar a archivo.parquet
#dfMovies.to_parquet(r'C:\Users\pc\Downloads\movies_dataset.parquet', engine = 'pyarrow')
dfMovies_parquet = pd.read_parquet(r'C:\Users\pc\Downloads\movies_dataset.parquet')
#corroborar la transformacion 
print(dfMovies_parquet.head())
print("Número de Registros CSV Original:", dfMovies.shape[0])
print("Número de Registros Parquet Transformado:", dfMovies_parquet.shape[0])